In [5]:
import numpy as np
import random

# Load data from TSPDATA.txt
def load_cities(filename):
    cities = []
    with open(filename, 'r') as f:
        lines = f.readlines()[2:]  # Skip the first two lines
        for line in lines:
            _, x, y = line.strip().split()
            cities.append((float(x), float(y)))
    return np.array(cities)

# Calculate distance matrix
def calculate_distance_matrix(cities):
    num_cities = len(cities)
    dist_matrix = np.zeros((num_cities, num_cities))
    for i in range(num_cities):
        for j in range(num_cities):
            dist_matrix[i, j] = np.sqrt(np.sum((cities[i] - cities[j]) ** 2))
    return dist_matrix

# Fitness function: inverse of total distance
def calculate_fitness(route, dist_matrix):
    total_distance = sum(dist_matrix[route[i], route[i+1]] for i in range(len(route) - 1))
    total_distance += dist_matrix[route[-1], route[0]]  # Return to start
    return 1 / total_distance

# Selection methods
def roulette_wheel_selection(population, fitness):
    total_fitness = sum(fitness)
    probs = [f / total_fitness for f in fitness]
    return population[np.random.choice(len(population), p=probs)]

def linear_rank_selection(population, fitness):
    ranked = sorted(range(len(fitness)), key=lambda i: fitness[i], reverse=True)
    ranks = [len(fitness) - rank for rank in range(len(fitness))]
    total_rank = sum(ranks)
    probs = [rank / total_rank for rank in ranks]
    return population[np.random.choice(len(population), p=probs)]

# Crossover methods
def edge_recombination_crossover(parent1, parent2):
    size = len(parent1)
    edge_map = {i: set() for i in parent1}

    # Build edge map from both parents
    for p in [parent1, parent2]:
        for i in range(size):
            current = p[i]
            next_city = p[(i + 1) % size]
            prev_city = p[(i - 1) % size]
            edge_map[current].update([next_city, prev_city])

    # Start building the child
    child = []
    current = random.choice(parent1)
    while len(child) < size:
        child.append(current)
        # Remove current from all edge sets
        for edges in edge_map.values():
            edges.discard(current)

        # Choose next city
        if edge_map[current]:
            # Select the city with the fewest edges, to minimize the recombination
            next_city = min(edge_map[current], key=lambda x: len(edge_map[x]))
        else:
            # If no edges are available, select a random remaining city
            remaining_cities = set(parent1) - set(child)
            if not remaining_cities:
                break  # Break the loop if no remaining cities
            next_city = random.choice(list(remaining_cities))

        current = next_city

    # If any cities are still missing, add them to the child
    remaining_cities = set(parent1) - set(child)
    child.extend(list(remaining_cities))

    return child


def pmx_crossover(parent1, parent2):
    size = len(parent1)
    child = [-1] * size
    start, end = sorted(random.sample(range(size), 2))

    child[start:end] = parent1[start:end]

    for i in range(start, end):
        if parent2[i] not in child:
            pos = i
            while start <= pos < end:
                pos = parent2.index(parent1[pos])
            child[pos] = parent2[i]

    for i in range(size):
        if child[i] == -1:
            child[i] = parent2[i]

    return child

# Mutation methods
def swap_mutation(route):
    if len(route) < 2:  # Safety check
        return
    a, b = random.sample(range(len(route)), 2)
    route[a], route[b] = route[b], route[a]

def insert_mutation(route):
    if len(route) < 2:  # Safety check
        return
    a, b = random.sample(range(len(route)), 2)
    city = route.pop(a)
    route.insert(b, city)

# Genetic algorithm
def genetic_algorithm(cities, dist_matrix, population_size=100, generations=1000, crossover_method='pmx', mutation_method='swap', selection_method='roulette'):
    num_cities = len(cities)

    # Initialize population
    population = [random.sample(range(num_cities), num_cities) for _ in range(population_size)]

    best_route = None
    best_distance = float('inf')

    for generation in range(generations):
        # Calculate fitness
        fitness = [calculate_fitness(route, dist_matrix) for route in population]

        # Select next generation
        new_population = []
        for _ in range(population_size // 2):
            if selection_method == 'roulette':
                parent1 = roulette_wheel_selection(population, fitness)
                parent2 = roulette_wheel_selection(population, fitness)
            elif selection_method == 'rank':
                parent1 = linear_rank_selection(population, fitness)
                parent2 = linear_rank_selection(population, fitness)

            # Crossover
            if crossover_method == 'pmx':
                child1 = pmx_crossover(parent1, parent2)
                child2 = pmx_crossover(parent2, parent1)
            elif crossover_method == 'edge':
                child1 = edge_recombination_crossover(parent1, parent2)
                child2 = edge_recombination_crossover(parent2, parent1)

            # Mutation
            if mutation_method == 'swap':
                swap_mutation(child1)
                swap_mutation(child2)
            elif mutation_method == 'insert':
                insert_mutation(child1)
                insert_mutation(child2)

            new_population.extend([child1, child2])

        population = new_population

        # Track best route
        for route in population:
            distance = 1 / calculate_fitness(route, dist_matrix)
            if distance < best_distance:
                best_distance = distance
                best_route = route

        # Print progress
        if (generation + 1) % 100 == 0:
            print(f"Generation {generation + 1}: Best Distance = {best_distance}")

    return best_route, best_distance

# Main
def main():
    cities = load_cities("TSPDATA.txt")
    dist_matrix = calculate_distance_matrix(cities)

    # Test all 8 scenarios
    scenarios = [
        ('pmx', 'swap', 'roulette'),
        ('pmx', 'swap', 'rank'),
        ('pmx', 'insert', 'roulette'),
        ('pmx', 'insert', 'rank'),
        ('edge', 'swap', 'roulette'),
        ('edge', 'swap', 'rank'),
        ('edge', 'insert', 'roulette'),
        ('edge', 'insert', 'rank'),
    ]

    results = []
    for crossover, mutation, selection in scenarios:
        print(f"Running scenario: Crossover={crossover}, Mutation={mutation}, Selection={selection}")
        best_route, best_distance = genetic_algorithm(
            cities, dist_matrix, crossover_method=crossover, mutation_method=mutation, selection_method=selection
        )
        results.append((crossover, mutation, selection, best_distance))

    print("\nResults:")
    for crossover, mutation, selection, distance in results:
        print(f"Crossover={crossover}, Mutation={mutation}, Selection={selection}, Best Distance={distance}")

if __name__ == "__main__":
    main()


Running scenario: Crossover=pmx, Mutation=swap, Selection=roulette
Generation 100: Best Distance = 525104.6731923058
Generation 200: Best Distance = 525104.6731923058
Generation 300: Best Distance = 525104.6731923058
Generation 400: Best Distance = 525104.6731923058
Generation 500: Best Distance = 525104.6731923058
Generation 600: Best Distance = 525104.6731923058
Generation 700: Best Distance = 525104.6731923058
Generation 800: Best Distance = 525104.6731923058
Generation 900: Best Distance = 525104.6731923058
Generation 1000: Best Distance = 525104.6731923058
Running scenario: Crossover=pmx, Mutation=swap, Selection=rank
Generation 100: Best Distance = 549265.6527908368
Generation 200: Best Distance = 545004.8744002008
Generation 300: Best Distance = 541548.4001451628
Generation 400: Best Distance = 541548.4001451628
Generation 500: Best Distance = 541548.4001451628
Generation 600: Best Distance = 541548.4001451628
Generation 700: Best Distance = 541548.4001451628
Generation 800: Bes